In [1]:
import re
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

In [2]:
# 1. 데이터 불러오기
with open('korean-english-park.train.ko', 'r', encoding='utf-8') as f:
    kor_sentences = f.readlines()

with open('korean-english-park.train.en', 'r', encoding='utf-8') as f:
    eng_sentences = f.readlines()

In [3]:
kor_sentences

['개인용 컴퓨터 사용의 상당 부분은 "이것보다 뛰어날 수 있느냐?"\n',
 '모든 광마우스와 마찬가지 로 이 광마우스도 책상 위에 놓는 마우스 패드를 필요로 하지 않는다.\n',
 '그러나 이것은 또한 책상도 필요로 하지 않는다.\n',
 '79.95달러하는 이 최첨단 무선 광마우스는 허공에서 팔목, 팔, 그외에 어떤 부분이든 그 움직임에따라 커서의 움직임을 조절하는 회전 운동 센서를 사용하고 있다.\n',
 '정보 관리들은 동남 아시아에서의 선박들에 대한 많은 (테러) 계획들이 실패로 돌아갔음을 밝혔으며, 세계 해상 교역량의 거의 3분의 1을 운송하는 좁은 해로인 말라카 해협이 테러 공격을 당하기 쉽다고 경고하고 있다.\n',
 '이 지역에 있는 미국 선박과 상업용 선박들에 대한 알카에다의 (테러) 시도 중 여러 건이 실패했다는 것을 알게 된 후에, 전문가들은 테러 조직이 여전히 세계 경제에 타격을 입히려 한다고 경고하고 있으며, 동남 아시아에 있는 세계 경제의 주요 통로가 위험에 처해 있다고 그들은 생각하고 있다.\n',
 '국립 과학 학회가 발표한 새 보고서에따르면, 복잡한 임무를 수행해야 하는 군인들이나 보다 오랜 시간 동안 경계를 늦추지 않고 있기 위해 도움이 필요한 군인들에게 카페인이 반응 시간을 증가시키고 임무 수행 능력을 향상시키는데 도움이 된다고 한다.\n',
 '이 보고서에따르면, "특히, 군사 작전에서 생사가 걸린 상황이 될 수도 있는 반응 속도와 시각 및 청각의 경계 상태를 유지시키기 위해 카페인이 사용될 수도 있다." 고 한다.\n',
 '"결정적인 순간에 그들의 능력을 증가시켜 줄 그 무엇이 매우 중요합니다."\n',
 '연구가들이 이미 커피 대체품으로서 음식 대용 과자나 껌에 카페인을 첨가하는 방법을 연구하고 있다고 Archibald는 말했다.\n',
 '약 200600밀리그램의, 비슷한 분량의 카페인은 또한 육체적 지구력을 강화시키는 데 효과적인 것 같으며, 특히 고도가 높은 곳에서 약해진 육체적 지구력을 일부 회복시켜주는 데 

In [4]:
eng_sentences

['Much of personal computing is about "can you top this?"\n',
 'so a mention a few weeks ago about a rechargeable wireless optical mouse brought in another rechargeable, wireless mouse.\n',
 "Like all optical mice, But it also doesn't need a desk.\n",
 'uses gyroscopic sensors to control the cursor movement as you move your wrist, arm, whatever through the air.\n',
 "Intelligence officials have revealed a spate of foiled plots on ships in Southeast Asia and are warning that a narrow stretch of water carrying almost one third of the world's maritime trade is vulnerable to a terror attack.\n",
 "After learning of several foiled al Qaeda attempts on U.S. and commercial ships in the area, experts are warning that the terror network still wants to cripple the global economy, the world's economic jugular vein in Southeast Asia is at risk.\n",
 'Caffeine can help increase reaction time and improve performance for military servicemen who must perform complex tasks or who need help staying aler

In [5]:
# 2. 중복 제거
data_pairs = list(set(zip(kor_sentences, eng_sentences)))

In [6]:
# 3. 전처리 함수 정의
def preprocess_kor(sentence):
    sentence = re.sub(r"[^가-힣\s]", "", sentence)
    return sentence.strip()

def preprocess_eng(sentence):
    sentence = re.sub(r"[^a-zA-Z\s]", "", sentence)
    sentence = '<start> ' + sentence.strip() + ' <end>'
    return sentence

In [7]:
kor_sentences = [preprocess_kor(sent) for sent, _ in data_pairs]
eng_sentences = [preprocess_eng(sent) for _, sent in data_pairs]

In [8]:
# 4. 토큰화
mecab = Mecab()
kor_sentences = [mecab.morphs(sent) for sent in kor_sentences]
eng_sentences = [sent.split() for sent in eng_sentences]

In [9]:
# 5. 데이터 필터링
kor_eng_pairs = [(kor, eng) for kor, eng in zip(kor_sentences, eng_sentences) if len(kor) <= 40 and len(eng) <= 40]

kor_corpus, eng_corpus = zip(*kor_eng_pairs)

In [10]:
# 6. 텐서 변환 및 토크나이저 생성
def tokenize(lang):
    tokenizer = Tokenizer(filters='')
    tokenizer.fit_on_texts(lang)
    tensor = tokenizer.texts_to_sequences(lang)
    tensor = pad_sequences(tensor, padding='post')
    return tensor, tokenizer

In [11]:
input_tensor, inp_tokenizer = tokenize(kor_corpus)
target_tensor, targ_tokenizer = tokenize(eng_corpus)

In [12]:
input_tensor

array([[ 1055,   541,  3762, ...,     0,     0,     0],
       [   60,   721,  2449, ...,     0,     0,     0],
       [12655,  3242,  2450, ...,     0,     0,     0],
       ...,
       [    2,     7,   324, ...,     0,     0,     0],
       [   34,    81,     7, ...,     0,     0,     0],
       [ 3693,  1460,    22, ...,     0,     0,     0]], dtype=int32)

In [13]:
target_tensor

array([[   2,   68, 6802, ...,    0,    0,    0],
       [   2,    1,  585, ...,    0,    0,    0],
       [   2,   39,   13, ...,    0,    0,    0],
       ...,
       [   2, 2948,   21, ...,    0,    0,    0],
       [   2,   92,   19, ...,    0,    0,    0],
       [   2, 2255,  715, ...,    0,    0,    0]], dtype=int32)

In [14]:
BUFFER_SIZE = len(input_tensor)
print(BUFFER_SIZE)
BATCH_SIZE = 64
steps_per_epoch = BUFFER_SIZE // BATCH_SIZE
print(steps_per_epoch)

67350
1052


In [15]:
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_tokenizer.word_index) + 1
vocab_tar_size = len(targ_tokenizer.word_index) + 1

In [16]:
dataset = tf.data.Dataset.from_tensor_slices((input_tensor, target_tensor)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [17]:
dataset

<BatchDataset shapes: ((64, 40), (64, 40)), types: (tf.int32, tf.int32)>

In [18]:
# 7. 모델 설계
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)  #입력, 출력 
        self.gru = tf.keras.layers.GRU(self.enc_units,  #은닉 상태의 차원 수 (enc_units)
                                       return_sequences=True, #출력 시퀀스의 각 타임스텝에서 출력 반환
                                       return_state=True,  #최종 은닉 상태 반환 
                                       recurrent_initializer='glorot_uniform')  #내부 가중치 초기화

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state

    def initialize_hidden_state(self):
        return tf.zeros((self.batch_sz, self.enc_units))

In [19]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)  #for query
        self.W2 = tf.keras.layers.Dense(units)  #for values
        self.V = tf.keras.layers.Dense(1)       

    def call(self, query, values):
        hidden_with_time_axis = tf.expand_dims(query, 1)  #(batch_size, 1, hidden_size) 시간 축 추가 
        score = self.V(tf.nn.tanh(
            self.W1(hidden_with_time_axis) + self.W2(values)))  #(batch_size, seq_len, hidden_size)는 values #(batch_size, seq_len, units)가 반환 
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1) #최종 context vector 생성 
        return context_vector, attention_weights


In [20]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units  #GRU 레이어의 은닉 상태 크기 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units, 
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output): 
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, state, attention_weights

In [21]:
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)


In [22]:
optimizer = tf.keras.optimizers.Adam()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [23]:
import os
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [24]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0
    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([targ_tokenizer.word_index['<start>']] * BATCH_SIZE, 1)
        for t in range(1, targ.shape[1]):
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions)
            dec_input = tf.expand_dims(targ[:, t], 1)
    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    return batch_loss

In [25]:
EPOCHS = 3

for epoch in range(EPOCHS):
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0
    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
    print(f'Epoch {epoch+1} Loss {total_loss/steps_per_epoch:.4f}')

Epoch 1 Loss 3.4834
Epoch 2 Loss 3.0440
Epoch 3 Loss 2.8250


In [ ]:
회고

- 시간이 너무 오래 걸렸다. 아이펠 끝나고 좀 더 학습시키고 결과를 봐야할 거 같다. 
- 그리고 아직 seq 2 seq attention모델이 익숙하지가 않다. 그래서 코드를 긁어오긴 했지만 이해가 잘 안간다. ...